# Linguistic Analyses for Compositional Abstractions
## Notebook 3: Forming conventions to talk about shared abstractions

This notebook was written by Robert Hawkins.  
Original modeling by Robert Hawkins, Will McCarthy, Cameron Holdaway, Haoliang Wang, and Judy Fan.

### *NOTE: THIS NOTEBOOK SERVES AS THE INSTRUCTOR VERSION*

In the previous notebook we inferred libraries of program fragments that correspond to the collection of part concepts people have in their heads as they step through the trials. For each trial we also generated programs that use these abstractions to express each scene more efficiently. Now we tackle the challenge of communicating these programs to another person. 

This is where language comes in. Words allow us to communicate arbitrarily complex concepts. Assuming that Architects and Builders learn the concepts over a trial sequence, each concept that is learned (each program fragment) could, in principle, be assigned a new word or phrase, and conveyed directly through the Architects' instructions. Well, almost. There is no guarantee that the words the Architect chooses to pick out a new concept would invoke the same concept in someone else. There is actually uncertainty about how the Builder will interpret the Architects' instructions and this, we suggest, might change what the Architect chooses to say.

Our hypothesis is that Architects trade-off communicative *efficiency* with communicative *effectiveness*. While they generally want to say things concisely, if there is too much uncertainty about how the Builder will interpret their words, they will choose a less ambiguous (if more wordy) way of expressing the same information. Concretely, if the Architect wanted to say "build an L" but they thought that the Builder wouldn't get what "L" meant, they might spell out the steps to make an L-shaped tower instead.

This notebook is divided into 3 sections:
**Section 1** demonstrates how we represent lexicons-- mappings of program elements to words.  
**Section 2** shows how we can program Architect and Builder agents to make decisions based on their lexicons.  
**Section 3** shows how we can update velie

## Setup 

In [ ]:
import os
import sys

import functools
import itertools
import json

import pandas as pd
import numpy as np

from numpy.random import choice

In [ ]:
# import classes for our model
sys.path.append("../model/convention_formation/")
from distribution import *
from lexicon import *

### read program data

In [ ]:
d = pd.read_json('../data/model/programs_for_you/programs_ppt_1.json')

## Section 1: Lexicons

### Representing lexicons

First we need a way to represent the mapping of part concepts to words in the Architects' and Builders' heads. 

We have defined a BlockLexicon class in `lexicon.py`, which stores the mapping of program primitives (from the previous section) to words and phrases.

Let's take this class out for a drive. 

We initialize it with the primitives of the agent's DSL on a given trial and an (ordered) list of available lexemes.

In [ ]:
# lexemes for program abstractions learned in previous section
# in our example, only 5 distinct abstractions were learned, so we only need 5 additional lexemes
lexemes = ['blah', 'blab', 'bloop', 'bleep', 'floop'] 

In [ ]:
dsl = d['dsl'][10]
l = BlockLexicon(dsl, lexemes)
print(dsl[0], '->', l.dsl_to_language(dsl[0]))
print(dsl[10], '->', l.dsl_to_language(dsl[10]))
print(dsl[-1], '->', l.dsl_to_language(dsl[-1]))

and we can also go in the other direction, converting from language to programmatic "concepts".

In [ ]:
print('place a horizontal block. ->', l.language_to_dsl('place a horizontal block.'))
print('move to the left by 8 ->', l.language_to_dsl('move to the left by 8'))
print('place a blah. ->', l.language_to_dsl('place a blah.'))
print('place a flomp. ->', l.language_to_dsl('place a womp.'))

### Representing beliefs about lexicons

To model the Architect's word choice, we explicitly model their beliefs about the Builder's lexicon. We represent this as a distribution over a set of possible lexicons. For this example, we're going to manually construct a distribution as another dictionary (from lexicons to probabilities). (See `distribution.py` for the implementation if you're interested in how this works).

Because our focus is on modeling abstractions learned during the task, we assume that Architect Agents and Builder Agents can unambiguously communicate about the base DSL-- moving left and right and placing individual blocks. I.e. when an Architect wants a Builder to place a block they will always say "place a horizontal block", and the Builder will correctly interpret this utterance.

The only thing that varies across different lexicons is the words used for *learned* program fragments. In practice (for this example) only five of these were learned across all participants' trial sequences. The set of possible lexicons is therefore fully defined by the set of possible mappings from these fragments to the `lexemes` defined above.

In [ ]:
# construct full lexicon for all possible mappings
possible_lexicons = [BlockLexicon(dsl, list(mapping)) for mapping in itertools.permutations(lexemes)]

We define the Architects' and Builders' priors (their initial beliefs) as the uniform distribution over these lexicons:

In [ ]:
prior = UniformDistribution(possible_lexicons)

print('Example lexicon')

print('lexicon:', json.dumps(prior.support()[0], indent = 4))

print('P(lexicon) =', prior.score(prior.support()[0]))

We can also marginalize to look at values of any particular chunk:

In [ ]:
print('possible values of chunk_L : ', 
      json.dumps(prior.marginalize(lambda d : d['chunk_L']), indent = 4))

The equal probabilities for each expression tell us that the Agents think each expression is an equally good (or bad) translation of "chunk_L". This makes sense for this artificial langauge (where "bleeps" and "bloops" are used to refer to abstractions). Real people likely have strong priors about what words will mean (we can make quite a lot of sense of "build an L" before any shared experience), but starting with a uniform prior provides a good starting point.

## Section 2: Simulating Agents

Now we have a way of representing beliefs over lexicons, we can define our Architect and Builder agents.

Both maintain a set of possible lexicons, and belief distribution over the *other agent's* lexicon.

The **Architect** agent makes choices about what to say based on it's beliefs about how the **Builder** will interpret them.  

The **Builder** takes actions based on it's beliefs about what the **Architect's** utterances mean.

In [ ]:
class FixedAgent() :
    def __init__(self, role, trial) :
        self.role = role
        self.actions = trial['dsl']

        # initialize beliefs to uniform prior over lexicons
        self.possible_lexicons = set([BlockLexicon(self.actions, list(mapping)) 
                                      for mapping in itertools.permutations(lexemes)])
        
#         print(lexemes)
        self.beliefs = UniformDistribution(self.possible_lexicons)
        self.utterances = set(list(self.possible_lexicons)[0].values())
        
    def act(self, observation) :
        if self.role == 'architect' :
            # get P(utt | target) by marginalizing over lexicons 
            utt_dist = EmptyDistribution()
            for lexicon in self.beliefs.support() :
                utt_dist.update({lexicon.dsl_to_language(observation) : self.beliefs.score(lexicon)})
            return choice(a = [*utt_dist.support()], 
                          p = [utt_dist.score(u) for u in utt_dist.support()])

        if self.role == 'builder' :
            # get P(a | utt) by marginalizing over lexicons 
            action_dist = EmptyDistribution()
            for lexicon in self.beliefs.support() :
                action_dist.update({lexicon.language_to_dsl(observation) : self.beliefs.score(lexicon)})
            return choice(a = [*action_dist.support()], 
                          p = [action_dist.score(a) for a in action_dist.support()])

In [ ]:
architect = FixedAgent('architect', d.loc[0].to_dict())
print('architect choice: ', architect.act('h'))

builder = FixedAgent('builder', d.loc[0].to_dict())
print('builder choice: ', builder.act('place a horizontal block.'))

In [ ]:
architect = FixedAgent('architect', d.loc[0].to_dict())
print('architect choice: ', architect.act('chunk_L'))

### Running simulations

Now we have our agents, we need to run them forward through the trial sequence.

In [ ]:
alpha = 0.04 # weight that trades off between TODO

In [ ]:
def run_simulation() :
    output = pd.DataFrame({"utt": [], "response": [], "target_program": [], "target_length" : [], "acc": []})
    for i, trial in d.iterrows() :
        architect = FixedAgent('architect', trial)
        builder = FixedAgent('builder', trial)

        # architect selects which program representation to comunicate proportional to length
        possiblePrograms = list(trial['programs_with_length'].keys())
        possibleLengths = np.array(list(trial['programs_with_length'].values()))
        utilities = np.exp(-alpha * possibleLengths) / sum(np.exp(-alpha * possibleLengths))
        target_program = choice(a = possiblePrograms, p = utilities)

        # loop through steps of target program one at a time
        utts, responses, accs = [], [], []
        for step in target_program.split(' ') :
            utt = architect.act(step)
            response = builder.act(utt)
            utts.append(utt)
            responses.append(response)
            accs.append(1.0 * (response == step))

        output = pd.concat([output, pd.DataFrame({
            "trial": int(i),
            "utt": utts,
            "response": responses,
            "acc": accs,
            "target_program": target_program,
            "target_length" : trial['programs_with_length'][target_program],
        })])
    return output

run_0 = run_simulation()
display(run_0)

In [ ]:
# let's inspect the first trial
print('target program: \n', run_0.query('trial==0').loc[0,'target_program'])
run_0.query('trial==0')[['utt','response','acc']]

In [ ]:
# let's inspect the final trial
print('target program: \n', run_0.query('trial==11').loc[0,'target_program'])

run_0.query('trial==11')[['utt','response','acc']]

### <span style="color: orange"> Exercise: explore how accuracy changes </span>

Wait, why is the accuracy so bad? Well, our agents aren't actually *learning* -- they're continuing to use their initial uniform priors.

## Section 3: Simulating learning

To have our agents learn, we need to extend the agent class to do Bayesian inference.

Here we add an update_beliefs function, which performs a Bayesian update on the beliefs about the other agent's lexicon based on the outcome of the previous trial. Note that this update happens when the class is initialized, so really we're defining a new agent at each step with updated beliefs.

In [ ]:
class LearningAgent(FixedAgent) :
    def __init__(self, role, curr_trial, previous_trial_df) :
        super().__init__(role, curr_trial)
        combined_primitives = set().union(*previous_trial_df['dsl']) if not previous_trial_df.empty else self.actions
        self.possible_lexicons = set([BlockLexicon(set().union(combined_primitives), list(mapping)) 
                                      for mapping in itertools.permutations(lexemes)])
        self.utterances = set(list(self.possible_lexicons)[0].values())
        self.update_beliefs(previous_trial_df)

    def update_beliefs(self, previous_trial_df) :
        # Initialize posterior 
        posterior = EmptyDistribution()
        posterior.to_logspace()

        # for each data point, calculate the marginal likelihood under lexicon distribution
        # P(l | obs) = 1/Z * P(l) * \prod_{o \in obs} P(o | l)
        # log P(l|obs) = -log Z + log P(l) + \sum_{o \in obs} log P(o | l)
        for lexicon in self.beliefs.support() :
            prior_term = np.log(self.beliefs.score(lexicon))
            likelihood_term = 0
            for i, step in previous_trial_df.iterrows() :
                if self.role == 'builder' :
                    likelihood_term += np.log(self.A1(step.target, lexicon).score(step.utterance))
                elif self.role == 'architect' :
                    likelihood_term += np.log(self.B0(step.utterance, lexicon).score(step.response))
            posterior.update({lexicon : prior_term + likelihood_term})
        posterior.renormalize()
        posterior.from_logspace()
        self.beliefs = posterior
        
    def B0(self, utt, lexicon) :
        builder_dist = EmptyDistribution()
        for action in self.actions :
            builder_dist.update({action : 1 if action == lexicon.language_to_dsl(utt) else 0.01})
        builder_dist.renormalize()
        return builder_dist
        
    def A1(self, target, lexicon) :
        architect_dist = EmptyDistribution()
        for utt in self.utterances :
            architect_dist.update({utt : 1 if utt == lexicon.dsl_to_language(target) else 0.01})
        architect_dist.renormalize()
        return architect_dist

In [ ]:
def run_learning_simulation(verbose = False) :
    output = pd.DataFrame({"utterance": [], "response": [], "target": [], "full_program" : [], "target_length" : [], "dsl" : [], "acc": []})
    for i, trial in d.iterrows() :
        architect = LearningAgent('architect', trial, output) # create agent with updated beliefs
        builder = LearningAgent('builder', trial, output)     # create agent with updated beliefs
        
        # architect selects which program representation to comunicate proportional to length
        possiblePrograms = list(trial['programs_with_length'].keys())
        possibleLengths = np.array(list(trial['programs_with_length'].values()))
        utilities = np.exp(-alpha * possibleLengths) / sum(np.exp(-alpha * possibleLengths))
        target_program = choice(a = possiblePrograms, p = utilities)

        # loop through steps of target program one at a time
        target_steps, utts, responses, accs = [], [], [], []
        for step in target_program.split(' ') :
            utt = architect.act(step)
            response = builder.act(utt)
            target_steps.append(step)
            utts.append(utt)
            responses.append(response)
            accs.append(response == step)

        if verbose:
            print('trial', i)
            print(pd.DataFrame({'utts' : utts, 'responses' : responses, 'correct' : accs, 'target' : target_steps}))
            print('beliefs about chunk_C meaning', 
                  json.dumps(architect.beliefs.marginalize(lambda d : d['chunk_C'] if 'chunk_C' in d else None), indent = 4))
        
        output = pd.concat([output, pd.DataFrame({
            "trial": i,
            "utterance": utts,
            "response": responses,
            "acc": accs,
            "target" : target_steps,
            "full_program": target_program,
            "dsl" : [trial['dsl']] * len(utts),
            "target_length" : trial['programs_with_length'][target_program],
        })])
    return output

learning_run_0 = run_learning_simulation()

In [ ]:
learning_run_0

### Jointly choose program and utterance

Wow, this is great! We can see our agents are updating their beliefs about the lexicon over time and able to get somewhat more accurate as they coordinate. But one of the most interesting things about our empirical data is that speakers seem to be strategically choosing which representation of the tower to convey -- our best current theory of why participants reduce the length of their utterances over time is that even when new library chunks come online, architects don't always try to refer to them right away. They aren't confident enough that their partner will understand, as the block-level descriptions are much safer. However, the block-level descriptions are also much *costlier* in terms of time and effect because they have to laboriously describe one action at a time. 

So far, we just used a placeholder for how the speaker picks which representation to communication: they just randomly pick from the list of candidates, slightly preferring shorter programs. However, there are other considerations that ought to go into this decision, namely the estimated likelihood that the listener will do the right thing.

As an exercise, add one line of code to the simulation to weight the target program according to its utility

In [ ]:
def run_strategic_simulation() :
    output = pd.DataFrame({"utterance": [], "response": [], "target": [], "full_program" : [], "target_length" : [], "dsl" : [], "acc": []})
    for i, trial in d.iterrows() :
        architect = LearningAgent('architect', trial, output)
        builder = LearningAgent('builder', trial, output)
        
        # architect selects which program representation to comunicate proportional to length
        possiblePrograms = list(trial['programs_with_length'].keys())
        possibleLengths = np.array(list(trial['programs_with_length'].values()))
        
        ## 
        ## utilities = np.exp(-alpha * possibleLengths) / sum(np.exp(-alpha * possibleLengths))
        ## 
        
        target_program = choice(a = possiblePrograms, p = utilities)

        # loop through steps of target program one at a time
        target_steps, utts, responses, accs = [], [], [], []
        for step in target_program.split(' ') :
            utt = architect.act(step)
            response = builder.act(utt)
            target_steps.append(step)
            utts.append(utt)
            responses.append(response)
            accs.append(response == step)

        print('trial', i)
        print(pd.DataFrame({'utts' : utts, 'responses' : responses, 'correct' : accs, 'target' : target_steps}))
        print('beliefs about chunk_C meaning', 
              json.dumps(architect.beliefs.marginalize(lambda d : d['chunk_C'] if 'chunk_C' in d else None), indent = 4))
        output = pd.concat([output, pd.DataFrame({
            "utterance": utts,
            "response": responses,
            "acc": accs,
            "target" : target_steps,
            "full_program": target_program,
            "dsl" : [trial['dsl']] * len(utts),
            "target_length" : trial['programs_with_length'][target_program],
        })])
    return output
display(run_learning_simulation())

### Summary


Congratulations! You've finished the tutorial!

We hope you found it fun and informative.